In [5]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import KFold, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from datetime import datetime
import xgboost as xgb

import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.metrics import mean_squared_error
from datetime import datetime
import math
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
# Set random seed for reproducibility
random_seed = 42

# Set the seed for numpy
np.random.seed(random_seed)

In [6]:
def processing_X(data):
    # Convert 'actual_offblock_time' and 'arrival_time' to 24-hour format (extract the hour)
    data['actual_offblock_hour'] = pd.to_datetime(data['actual_offblock_time']).dt.hour
    data['arrival_hour'] = pd.to_datetime(data['arrival_time']).dt.hour
    
    # Convert 'date' to the month
    data['month'] = pd.to_datetime(data['date']).dt.month
    
    # Define the features and target variable
    X = data.drop(['tow', 'flight_id', 'name_adep','name_ades', 'callsign', 'actual_offblock_time', 'arrival_time', 'date', 'takeoff_groundspeed', 'takeoff_time_duration'], axis=1)
    y = data['tow']
    
    # Categorical and numerical features
    categorical_features = ['adep', 'ades', 'country_code_adep', 'country_code_ades',
                            'aircraft_type', 'wtc', 'airline']
    numerical_features = ['flight_duration', 'taxiout_time', 'flown_distance', 'actual_offblock_hour',
                          'arrival_hour', 'month', 'highest_altitude','cruising_altitude', 'time_to_reach_altitude_mode',
                          'average_u_component_of_wind', 'average_v_component_of_wind', 'average_temperature', 'flight_dist']

    
    # Define the preprocessor
    # Define the preprocessor
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numerical_features),
            ('cat', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), categorical_features)
        ])
    X_trans = preprocessor.fit_transform(X)
    return X_trans, preprocessor

In [7]:
def process_input_file(df):
    df['actual_offblock_hour'] = pd.to_datetime(df['actual_offblock_time']).dt.hour
    df['arrival_hour'] = pd.to_datetime(df['arrival_time']).dt.hour
    df['month'] = pd.to_datetime(df['date']).dt.month
    return df

In [9]:
# Load your training dataset (with both features and 'tow' column)
# Replace 'train_dataset.csv' with the actual path to your training dataset
train_data = pd.read_csv('../challenge_set_update_v1.csv')
train_data = process_input_file(train_data)
test_data = pd.read_csv('../submission_set_update_v1.csv')
test_data = process_input_file(test_data)

all_data = pd.concat([train_data, test_data], ignore_index=True)

len_train = len(train_data)

# Procssing X 
X_input, preprocessor = processing_X(all_data)

X_train = X_input[:len_train, :]
predict_data = X_input[len_train:, :]

y_train = train_data['tow']  # Target variable (tow)

# Store the 'flight_id' from the raw data
flight_ids = test_data['flight_id']
aircraft_types = test_data['aircraft_type']

# Initialize the XGBoost Regressor
xg_reg = xgb.XGBRegressor(objective='reg:squarederror', colsample_bytree=0.3, learning_rate=0.1,
                          max_depth=5, alpha=10, n_estimators=300)

# Train the model
xg_reg.fit(X_train, y_train)

# Predict on the new dataset (without 'tow' column)
y_pred = xg_reg.predict(predict_data)

# Save the predictions to a CSV file
output = pd.DataFrame({'flight_id': flight_ids, 'aircraft_type': aircraft_types, 'tow_predicted': y_pred})

prcdata_mtow_etow_df = pd.read_csv('../PRCData_MTOW_ETOW.csv')

# Merge the two dataframes based on the aircraft type
merged_df = pd.merge(output, prcdata_mtow_etow_df, left_on='aircraft_type', right_on='Aircraft', how='left')

# Calculate the 20% margin
merged_df['lower_limit'] = merged_df['OEW_kg']   # 20% lower than OEW
merged_df['upper_limit'] = merged_df['MTOW_kg']  # 20% higher than MTOW

# Now we clip the tow_predicted values between the MTOW and OEW values for each aircraft type
merged_df['tow'] = merged_df['tow_predicted'].clip(lower=merged_df['lower_limit'], upper=merged_df['upper_limit'])



# Select the relevant columns to display: flight_id, aircraft_type, tow_predicted, tow_clipped
result_df = merged_df[['flight_id', 'tow']]

result_df.to_csv('my_submission.csv', index=False)

print("Predictions saved to 'my_submission.csv'")


Predictions saved to 'my_submission_v1.csv'
